# 🧠 Predict Plant Diseases from Multi-Leaf Images (YOLOv8 + CNN)

In [4]:
import cv2
import numpy as np
import os
from ultralytics import YOLO
import tensorflow as tf
from tensorflow.keras.models import load_model

In [5]:
# Load models
cnn_model = load_model("trained_plant_disease_model.keras")
yolo_model = YOLO("yolov8n.pt")

In [6]:
# Configuration
CNN_INPUT_SIZE = (128, 128)
labels = ['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Blueberry___healthy', 'Cherry_(including_sour)___Powdery_mildew', 'Cherry_(including_sour)___healthy', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Corn_(maize)___Common_rust_', 'Corn_(maize)___Northern_Leaf_Blight', 'Corn_(maize)___healthy', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Grape___healthy', 'Orange___Haunglongbing_(Citrus_greening)', 'Peach___Bacterial_spot', 'Peach___healthy', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Raspberry___healthy', 'Soybean___healthy', 'Squash___Powdery_mildew', 'Strawberry___Leaf_scorch', 'Strawberry___healthy', 'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Target_Spot', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Tomato___Tomato_mosaic_virus', 'Tomato___healthy']

In [7]:
# Utility Functions
def preprocess(image):
    image = cv2.resize(image, CNN_INPUT_SIZE)
    image = image / 255.0
    image = np.expand_dims(image, axis=0)
    return image

def decode_prediction(pred):
    class_idx = np.argmax(pred)
    return labels[class_idx], float(pred[0][class_idx])

In [8]:
def detect_diseases_from_image(image_path):
    """
    Detects leaves/plants in the image and classifies disease for each.

    Returns:
        A list of dicts with 'leaf_index', 'disease', and 'confidence'.
    """
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Image not found: {image_path}")

    results = yolo_model(image)

    detected_diseases = []

    for i, box in enumerate(results[0].boxes.xyxy):
        x1, y1, x2, y2 = map(int, box)
        leaf_crop = image[y1:y2, x1:x2]

        if leaf_crop.size == 0:
            continue

        input_img = preprocess(leaf_crop)
        prediction = cnn_model.predict(input_img, verbose=0)
        label, confidence = decode_prediction(prediction)

        detected_diseases.append({
            'leaf_index': i + 1,
            'disease': label,
            'confidence': round(confidence, 4)
        })

    return detected_diseases

In [ ]:
result = detect_diseases_from_image("treating-brown-spots-on-leaves-5076039-01-b05c9ed7a44648adbee4c7d4bc2b2bab.jpg")
for r in result:
    print(f"Leaf {r['leaf_index']}: {r['disease']} ({r['confidence']*100:.1f}%)")


0: 448x640 1 cup, 3 cakes, 106.8ms
Speed: 8.0ms preprocess, 106.8ms inference, 2.4ms postprocess per image at shape (1, 3, 448, 640)


Leaf 1: Corn_(maize)___healthy (9.6%)
Leaf 2: Corn_(maize)___healthy (9.1%)
Leaf 3: Corn_(maize)___healthy (8.9%)
Leaf 4: Squash___Powdery_mildew (9.0%)
